In [ ]:
news['noun'] = 0
news['propn'] = 0
news['verb'] = 0
news['adj'] = 0
news['adv'] = 0
news['num'] = 0

In [ ]:
def count_pos(row):
    doc = nlp(row.statement)
    for token in doc:
        try:
            row[token.pos_.lower()] += 1
        except:
            pass
        
    return row

In [ ]:
news = news.apply(count_pos,axis=1)

In [ ]:
doc = nlp(news.statement[1])

In [ ]:
news.statement[1]

In [ ]:
for token in doc:
    print (token)
#     for child in token.children:
#         print (child)
    print (get_depth(token))
#     print (token.pos_)
#     break

In [ ]:
subjects_count = {}
for subj in unique_subjects:
    subjects_count[subj] = subjects_list.count(subj)
subjects_count = {value[0]:value[1] for value in sorted(subjects_count.items(), key=lambda x: -x[1])}

In [ ]:
plt.figure(figsize=(8,12))
sns.barplot(x=list(subjects_count.values())[0:50], y=list(subjects_count.keys())[0:50])
plt.show()

In [ ]:
# ohe = OneHotEncoder(drop=[['education_official'],['Russia']])
ohe = OneHotEncoder()
ohe.fit(news_train[['state']])

In [ ]:
ohe.transform(news_train[['state']])[:,1:]

In [ ]:
ohe.transform(news_train[['state']])

In [ ]:
ohe.get_feature_names()

In [ ]:
ohe.categories_

In [ ]:
test_mlb = MultiLabelBinarizer(sparse_output=False)
test_sub = [set(value.split(',')) for value in news_train['subjects']]
test_mlb.fit(test_sub)
# test_sub

In [ ]:
test_sub2 = [set(value.split(',')) for value in news_valid['subjects']]
test_mlb.transform(test_sub2)

In [ ]:
test_sub3 = [set(value.split(',')) for value in news_test['subjects']]
test_mlb.transform(test_sub3)

In [ ]:
test_mlb.classes_

In [ ]:
test_pipeline = Pipeline([
    ('union', FeatureUnion(
        transformer_list=[

            # Pipeline to dummify party column
            ('party', Pipeline([
                ('selector', ItemSelector(key='party')),
                ('ohe', Dummify()),
            ])),
            
            # Pipeline to dummify context column
            ('context', Pipeline([
                ('selector', ItemSelector(key='context')),
                ('ohe', Dummify()),
            ])),
            
            # Pipeline to dummify state column
            ('state', Pipeline([
                ('selector', ItemSelector(key='state')),
                ('ohe', Dummify()),
            ])),
            
            # Pipeline to dummify subjects column
            ('subjects', Pipeline([
                ('selector', ItemSelector(key='subjects')),
                ('ohe', Dummify(multilabel=True)),
            ])),

        ],
    )),
])    

In [ ]:
test_pipeline.fit(news_train)

In [ ]:
test_pipeline.transform(news_train)

In [ ]:
test_pipeline.transform(news_valid)

In [ ]:
dict(test_pipeline.named_steps['union'].transformer_list).get('party').named_steps['ohe'].get_feature_names(['party'])

In [ ]:
dict(test_pipeline.named_steps['union'].transformer_list).get('context').named_steps['ohe'].get_feature_names(['context'])

In [ ]:
dict(test_pipeline.named_steps['union'].transformer_list).get('state').named_steps['ohe'].get_feature_names(['state'])

In [ ]:
dict(test_pipeline.named_steps['union'].transformer_list).get('subjects').named_steps['ohe'].get_feature_names(['subject'])

In [ ]:
dict(transformer_pipeline.named_steps['union'].transformer_list).get('text_stats').named_steps['dict_vect'].get_feature_names()